# Notebook to demonstrate the use of hierarchical category encoders

Load some important modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from category_encoders import *

We are going to use sklearn to predict a target

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Load the datasets:
1. The median house prices per ward in England and Wales (UK)
2. The lookup table for Wards to Local Authority District to County to Region to Country

The first dataset can be downloaded here:
https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/medianpricepaidbywardhpssadataset37

The xls spreadsheet has the following sheets:

* Cover sheet
* Contents
* Datasets
* Metadata
* Terms and Conditions
* 1a
* 1b
* 1c
* 1d
* 1e
* 2a
* 2b
* 2c
* 2d
* 2e
* 3a
* 3b
* 3c
* 3d
* 3e
* Related Publications

We need the 5th sheet: "1a".  I used the ssconvert command from gnumeric to export this sheet into xls:

> ssconvert --export-file-per-sheet /Users/lisa/Downloads/HPSSA\ Dataset\ 37\ -\ Median\ price\ paid\ by\ ward.xls HPSSA_Dataset_37.csv

and then trimmed the top and bottom of the csv to clean it:

> sed -e '1,5d; 8060,$d' HPSSA_Dataset_37_1a.csv > HPSSA_Dataset_37_1a.csv

The second dataset can be downloaded from here:
https://geoportal.statistics.gov.uk/search?collection=Dataset&sort=-created&tags=all(LUP_WD_LAD_CTY_RGN_GOR_CTRY)

In [3]:
# load the prices dataset

import pkg_resources

data_filename = "../data/HPSSA_Dataset_37_1a.csv"

stream = pkg_resources.resource_filename(__name__, data_filename)

with open(stream) as f:
    df = pd.read_csv(f, encoding='latin-1')
    
# Remove Incomplete rows
df = df[(df['Year ending Mar 2017'] != ":") & (df['Year ending Mar 2022'] != ":")]
# convert prices to numeric
df[['Year ending Mar 2017','Year ending Mar 2022']] = df[['Year ending Mar 2017','Year ending Mar 2022']].astype(float)
# remove LA code 'E06000053' as there is only one remaining value - will break the train/test split later
df = df[df['Local authority code'] != "E06000053"]
# remove "unnamed" columns:
df.drop(list(df.filter(regex = 'Unnamed:')), axis = 1, inplace = True)

In [4]:
# load the lookup table:

data_filename = "../data/Ward_lookup_table.csv"

stream = pkg_resources.resource_filename(__name__, data_filename)

with open(stream) as f:
    lookup = pd.read_csv(f, encoding='latin-1')
    
    
# Clean up the lookup - impute the NaNs with appropriate values (must be filled to make a dictionary later)
lookup['RGN20CD'].fillna(lookup['CTRY20CD'], inplace=True)
lookup['RGN20NM'].fillna(lookup['CTRY20NM'], inplace=True)
lookup['CTY20CD'].fillna(lookup['LAD20CD'], inplace=True)
lookup['CTY20NM'].fillna(lookup['LAD20NM'], inplace=True)

Add the additional geographic columns to the prices dataset:

In [5]:
df_merged = df.merge(lookup[['WD20CD','RGN20CD','RGN20NM','CTRY20CD','CTRY20NM']], left_on='Ward code', right_on='WD20CD')
df_merged.drop(['WD20CD'],axis=1,inplace=True)

In [6]:
df_merged.head()

,Local authority code,Local authority name,Ward code,Ward name,Year ending Dec 1995,Year ending Mar 1996,Year ending Jun 1996,Year ending Sep 1996,Year ending Dec 1996,Year ending Mar 1997,...,Year ending Dec 2020,Year ending Mar 2021,Year ending Jun 2021,Year ending Sep 2021,Year ending Dec 2021,Year ending Mar 2022,RGN20CD,RGN20NM,CTRY20CD,CTRY20NM
0,E06000001,Hartlepool,E05008945,Foggy Furze,39000,39000,39000,38625,38250,38250,...,107500,111250,111750,115000,115500,115500.0,E12000001,North East,E92000001,England
1,E06000001,Hartlepool,E05008946,Hart,56500,56500,56500,56950,58425,59950,...,160000,163000,166500,166000,166500,168000.0,E12000001,North East,E92000001,England
2,E06000001,Hartlepool,E05008947,Headland and Harbour,30950,33950,34975,33950,34950,30750,...,90000,90000,96250,90000,94750,97250.0,E12000001,North East,E92000001,England
3,E06000001,Hartlepool,E05008943,De Bruce,38000,38000,37000,30500,30000,30500,...,134500,120000,119500,114950,115000,97000.0,E12000001,North East,E92000001,England
4,E06000001,Hartlepool,E05008944,Fens and Rossmere,51000,51000,50000,47975,47500,47500,...,131000,136375,144000,145000,145500,149000.0,E12000001,North East,E92000001,England


Create a specific modelling dataset:

In [7]:
# Note we will not use the column 'Year ending Mar 2022' in the features - we just need it right now for the baseline!
X = df_merged[['Local authority code','Local authority name','Ward code','Ward name','RGN20CD','RGN20NM','CTRY20CD','CTRY20NM','Year ending Mar 2017','Year ending Mar 2022']]
y = df_merged['Year ending Mar 2022']

## Features

Check out the postcode column to identify how many unique postcodes we have:

In [8]:
print("Length of Countries:",len(X['CTRY20CD'].unique()))
print("Length of Region code:",len(X['RGN20CD'].unique()))
print("Length of LA code:",len(X['Local authority code'].unique()))
print("Length of Ward code:",len(X['Ward code'].unique()))

Length of Countries: 2
Length of Region code: 10
Length of LA code: 335
Length of Ward code: 8041


In [9]:
features = ['RGN20CD','CTRY20CD','Local authority code','Year ending Mar 2017']

## Create a train / test set and calculate a baseline

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=X['Local authority code'], random_state = 42)

Get a baseline prediction:

In [12]:
# Calculate the average prices within a Local Authority as a baseline map:
baseline_map = X_train.groupby('Local authority code')['Year ending Mar 2022'].mean().to_dict()

# Use the baseline price map to predict the test prices:
y_baseline = X_test['Local authority code'].map(baseline_map)

# Calculate baseline errors and display average baseline error
baseline_errors = abs(y_baseline - y_test)

print('Average baseline error: £', round(np.mean(baseline_errors), 2))

Average baseline error: £ 67461.65


In [13]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (baseline_errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 77.22 %.


So we have to beat this accuracy with our model!

For completeness, let's delete the target from X to be sure!

In [14]:
X_train.drop(['Year ending Mar 2022'], axis = 1, inplace = True)
X_test.drop(['Year ending Mar 2022'], axis = 1, inplace = True)

## Simple prediction using Random Forest

Ordinal encode the features:

In [15]:
enc = OrdinalEncoder(verbose=1).fit(X_train[features], y_train)
X_train_enc = enc.transform(X_train[features])
X_test_enc = enc.transform(X_test[features])

In [16]:
X_train_enc.head()
X_test_enc.head()

,RGN20CD,CTRY20CD,Local authority code,Year ending Mar 2017
4452,10,1,25,120000.0
2897,7,1,302,327750.0
1345,4,1,52,231995.0
6000,8,1,247,166500.0
4436,10,1,25,131250.0


Predict the target using a simple RF classifer on the encoded features:

In [17]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train_enc, y_train)
y_pred = rf.predict(X_test_enc)

In [18]:
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error: £', round(np.mean(errors), 2))

Mean Absolute Error: £ 30756.07


In [19]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 90.24 %.


In [20]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Year ending Mar 2017 Importance: 0.96
Variable: Local authority code Importance: 0.03
Variable: RGN20CD              Importance: 0.01
Variable: CTRY20CD             Importance: 0.0


## Prediction using hierarchical encoding

For this we will require a dictionary of the lookup hierarchy:

In [21]:
# Create a lookup dictionary of geography by codes:

lookup_short = lookup[['CTRY20CD', 'RGN20CD', 'LAD20CD']]

d = {k: f.groupby('RGN20CD')['LAD20CD'].unique().apply(tuple).to_dict()
        for k, f in lookup_short.groupby('CTRY20CD')
          for kk, ff in lookup_short.groupby('RGN20CD')
              }

In [22]:
hier_cols = ['Local authority code']
cols = ['Local authority code', 'Year ending Mar 2017']


henc = TargetEncoder(verbose=1, hierarchy=d, cols=hier_cols).fit(X_train[cols], y_train)
X_train_henc = henc.transform(X_train[cols])
X_test_henc = henc.transform(X_test[cols])

/opt/anaconda3/envs/catenc/lib/python3.10/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/opt/anaconda3/envs/catenc/lib/python3.10/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [23]:
X_train_henc = X_train_henc.rename({'Local authority code': 'Local authority code HE'}, axis=1)
X_test_henc = X_test_henc.rename({'Local authority code': 'Local authority code HE'}, axis=1)

In [24]:
X_train_henc = pd.concat([X_train_henc, X_train_enc[['RGN20CD','CTRY20CD','Local authority code']]], axis=1)
X_test_henc = pd.concat([X_test_henc, X_test_enc[['RGN20CD','CTRY20CD','Local authority code']]], axis=1)

In [25]:
X_train_henc.head()

,Local authority code HE,Year ending Mar 2017,RGN20CD,CTRY20CD,Local authority code
7068,817506.057673,540000.0,1,1,1
2530,556810.538458,335000.0,2,1,2
205,209436.904961,194750.0,3,1,3
1313,279712.917526,230000.0,4,1,4
424,200896.687511,244500.0,5,1,5


Predict using the new hierarchical encoded feature:

In [26]:
he_rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
he_rf.fit(X_train_henc, y_train)
y_pred_he = he_rf.predict(X_test_henc)

In [27]:
# Calculate the absolute errors
errors_he = abs(y_pred_he - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error: £', round(np.mean(errors_he), 2))

Mean Absolute Error: £ 30371.34


In [28]:
# Calculate mean absolute percentage error (MAPE)
mape_he = 100 * (errors_he / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape_he)

print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 90.37 %.


In [29]:
# Get numerical feature importances
importances_he = list(he_rf.feature_importances_)

# List of tuples with variable and importance
feature_importances_he = [(feature, round(importance, 2)) for feature, importance in zip(X_train_henc.columns, importances_he)]
# Sort the feature importances by most important first
feature_importances_he = sorted(feature_importances_he, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances_he];

Variable: Year ending Mar 2017 Importance: 0.95
Variable: Local authority code HE Importance: 0.02
Variable: Local authority code Importance: 0.02
Variable: RGN20CD              Importance: 0.01
Variable: CTRY20CD             Importance: 0.0
